<a href="https://colab.research.google.com/github/Georgi-Petkov/DS_Portfolio/blob/master/Datapane_Stock_Market_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Datapane Tutorial - Stock Analysis

---

This is part of a series of tutorials to help you learn how to use Datapane, and was built by _Kaan Şimşek_.

Before getting started, you will need your API token, which you can find on your [settings page](https://datapane.com/settings). If you are on a private Datapane instance, you will find it in `/settings` on your instance domain (i.e., https://[your-instance].datapane.net/settings).

Once you have you token, add it to the form field in the cell below to login to Datapane.

If you are using Datapane Community, set the `datapane_server_url` as `https://datapane.com`. Alternatively, if you are using a Teams instance, enter the URL of your instance (i.e. https://[your-instance].datapane.net).


In [ ]:
datapane_api_token = "" #@param {type:"string"}
datapane_server_url = "https://datapane.com" #@param {type:"string"}

In [ ]:
!pip3 install --quiet datapane 

import datapane as dp
dp.login(token=datapane_api_token, server=datapane_server_url) # Make sure you've set your API token above!

     |████████████████████████████████| 1.2MB 4.0MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
     |████████████████████████████████| 174kB 39.9MB/s 
     |████████████████████████████████| 645kB 41.2MB/s 
     |████████████████████████████████| 102kB 9.7MB/s 
     |████████████████████████████████| 92kB 9.6MB/s 
     |████████████████████████████████| 61kB 6.5MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
ERROR: nbclient 0.5.3 has requirement jupyter-client>=6.1.5, but you'll have jupyter-client 5.3.5 which is incompatible.
Connected successfully to https://acme.datapane.net as leo


'leo'

# Import libraries and pull data

In [ ]:
import pandas as pd
import numpy as np
import plotly.figure_factory as ff
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px
import pandas_datareader.data as web
import datetime

import plotly.io as pio
pio.templates.default = "ggplot2"

start = datetime.datetime(2019, 8, 1)
end = datetime.datetime.now()

zm = web.DataReader("CSCO", 'yahoo', start, end).reset_index()
nflx = web.DataReader("NFLX", 'yahoo', start, end).reset_index()


# Building Plotly visualizations

First, let's create our plots.

In [ ]:
trace0 = go.Scatter(x=nflx.Date, y=nflx.Close, name='nflx')
fig0 = go.Figure([trace0])
fig0.update_layout(
    title={
        'text': "Netflix Stock Price",
        'x':0.5,
        'xanchor': 'center'})
fig0.show()

In [ ]:
nflx['10-day MA'] = nflx['Close'].rolling(window=10).mean()
nflx['20-day MA'] = nflx['Close'].rolling(window=20).mean()
nflx['50-day MA'] = nflx['Close'].rolling(window=50).mean()

trace0 = go.Scatter(x=nflx.Date, y=nflx.Close, name='NFLX')
trace1 = go.Scatter(x=nflx.Date, y=nflx['10-day MA'], name='10-day MA')
trace2 = go.Scatter(x=nflx.Date, y=nflx['20-day MA'], name='20-day MA')
fig1 = go.Figure([trace0, trace1, trace2])
fig1.update_layout(
    title={
        'text': "Netflix Stock Price",
        'x':0.5,
        'xanchor': 'center'})
fig1.show()

In [ ]:
fig2 = go.Figure(go.Candlestick(x=nflx.Date, open=nflx.Open, high=nflx.High, low=nflx.Low, close=nflx.Close))
fig2.update_layout(
    title={
        'text': "Netflix Stock Price (Candle Stick)",
        'x':0.5,
        'xanchor': 'center'})
fig2.show()

In [ ]:
nflx['Daily return (%)'] = round(nflx['Close'].pct_change()*100, 2)

fig3 = px.bar(nflx, x="Date", y="Daily return (%)")
fig3.update_layout(
    title={
        'text': "Netflix Stock Daily Return",
        'x':0.5,
        'xanchor': 'center'})
fig3.show()

In [ ]:
trace0 = go.Scatter(x=nflx.Date, y=nflx.Close, name='NFLX', line=dict(color='lime'))
trace1 = go.Scatter(x=zm.Date, y=zm.Close, name='ZM', line=dict(color='red'))

fig4 = go.Figure([trace0, trace1])
fig4.update_layout(dict(
    title=dict({
        'text': "Netflix vs Zoom Stock Price",
        'x':0.5,
        'xanchor': 'center'}),
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(count=3,
                     label='3m',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(count=12,
                     label='12m',
                     step='month',
                     stepmode='backward')
            ])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date')))
fig4.show()

In [ ]:
trace0 = go.Scatterpolar(
      r = [nflx['Close'].mean(),nflx['Open'].min(), nflx['Low'].min(),nflx['High'].max()],
      theta = ['Close','Open','Low','High'], line =  dict(
            color = 'lime'), name='NFLX',
      fill = 'toself')

trace1 = go.Scatterpolar(
      r = [zm['Close'].mean(),zm['Open'].min(), zm['Low'].min(),zm['High'].max()],
      theta = ['Close','Open','Low','High'], line =  dict(
            color = 'red'), name='ZM',
      fill = 'toself'
    )

fig5 = go.Figure([trace0,trace1])
fig5.update_layout(go.Layout(
      polar = dict(
        radialaxis = dict(
          visible = True)),
            title=dict({
        'text': "Netflix vs Zoom Stock Price (Radar Chart)",
        'x':0.5,
        'xanchor': 'center'})))
fig5.show()

In [ ]:
df = web.DataReader(['BABA', 'FB', 'AAPL', 'SPOT'], 'yahoo', start, end)['Close']

trace0 = go.Box(y=df.BABA, name='BABA', line=dict(color='blue'))
trace1 = go.Box(y=df.FB, name='FB', line=dict(color='red'))
trace2 = go.Box(y=df.AAPL, name='AAPL', line=dict(color='green'))
trace3 = go.Box(y=df.SPOT, name='SPOT', line=dict(color='purple'))

fig6 = go.Figure([trace0, trace1, trace2, trace3])
fig6.update_layout(
    title={
        'text': "Stock Prices",
        'x':0.5,
        'xanchor': 'center'})
fig6.show()

In [ ]:
nflx = nflx[['Open','Close','Volume']]
nflx["index"] = np.arange(len(nflx))

fig7 = go.Figure(ff.create_scatterplotmatrix(nflx, diag='box', index='index',size=3,
                               height=600, width=1150, colormap='RdBu',
                                   title={
                                    'text': "Netflix Stock Price (Scatterplot Matrix)",
                                    'x':0.5,
                                    'xanchor': 'center'}))
fig7.show()

# Building a Datapane report

Now that we have a series of plots created using Plotly, we can create a report using Datapane. 

In addition to the visualizations, this report includes Datapane's `BigNumber` component to display today's stock prices, and our `DataTable` component to allow our viewers to filter, explore, and download the data themselves.

Although we could create a linear list of all blocks (similar to how they are displayed in this notebook), we can build a more powerful and accessible report by using Datapane's [layout components](https://docs.datapane.com/reports/blocks/layout-pages-and-selects). 

From these components, we are using `Group` to place the `BigNumber` blocks in two columns, have used the `Page` block to add multiple pages to our report, and are using the `Select` block to create tabs where users can choose their dataset. This results in a beautiful, interactive document which we can publish and share.


In [ ]:
netflix_today = nflx.iloc[-1] 
zm_today = zm.iloc[-1]

dp.Report(
    dp.Page(
        title='Dashboard',
        blocks=[
            dp.Group(
                dp.BigNumber(
                    heading="Netflix Day Performance",
                    value="${:,.2f}".format(netflix_today.Close),
                    prev_value="${:,.2f}".format(netflix_today.Open),
                ), 
                dp.BigNumber(
                    heading="Zoom Day Performance",
                    value="${:,.2f}".format(zm_today.Close),
                    prev_value="${:,.2f}".format(zm_today.Open),
                ),        
                columns=2 
            ),
            dp.Group(
                fig0, fig1, fig2, fig3, fig4, fig5,
                columns=2, 
            ),
            fig6, fig7
      ]),
      dp.Page(title='Datasets', blocks=[
          "_These datasets are pulled live when the report is created_.",                  
          dp.Select(
              dp.Table(nflx, label='Netflix'), 
              dp.Table(zm, label='Zoom'), 
              dp.Table(df, label='Comparisons')
          )
      ])
).save(path='html_report.html', open=True)

Thanks for using Datapane, to automate and securely share documents in your organization please see Datapane Cloud - https://datapane.com/
